In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Analyzing London Crime data").getOrCreate()

24/04/08 23:56:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
### Reading Data in spark data frame

data = spark.read\
            .format("csv")\
            .option("header","true")\
            .load("/Volumes/T7/GettingStartedSpark2/london_crime_by_lsoa.csv")

In [6]:
data.printSchema()

root
 |-- lsoa_code: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- major_category: string (nullable = true)
 |-- minor_category: string (nullable = true)
 |-- value: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)



In [8]:
### Limit(5) -> shows top 5 results

data.limit(5).show()

+---------+----------+--------------------+--------------------+-----+----+-----+
|lsoa_code|   borough|      major_category|      minor_category|value|year|month|
+---------+----------+--------------------+--------------------+-----+----+-----+
|E01001116|   Croydon|            Burglary|Burglary in Other...|    0|2016|   11|
|E01001646| Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|E01000677|   Bromley|Violence Against ...|      Other violence|    0|2015|    5|
|E01003774| Redbridge|            Burglary|Burglary in Other...|    0|2016|    3|
|E01004563|Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
+---------+----------+--------------------+--------------------+-----+----+-----+



In [9]:
data.dropna()

DataFrame[lsoa_code: string, borough: string, major_category: string, minor_category: string, value: string, year: string, month: string]

In [10]:
data = data.drop("lsoa_code")
## Dropping data which is not useful

In [12]:
total_buroughs = data.select('borough').distinct()
total_buroughs.show()

### Distinct shows the unique data in the column

+--------------------+
|             borough|
+--------------------+
|             Croydon|
|          Wandsworth|
|              Bexley|
|             Lambeth|
|Barking and Dagenham|
|              Camden|
|           Greenwich|
|              Newham|
|       Tower Hamlets|
|            Hounslow|
|              Barnet|
|              Harrow|
|Kensington and Ch...|
|           Islington|
|               Brent|
|            Haringey|
|             Bromley|
|              Merton|
|         Westminster|
|             Hackney|
+--------------------+
only showing top 20 rows



In [13]:
### Filtering data -> filter(cond)

hackney_data = data.filter(data["borough"]=="Hackney")
hackney_data.show()

+-------+--------------------+--------------------+-----+----+-----+
|borough|      major_category|      minor_category|value|year|month|
+-------+--------------------+--------------------+-----+----+-----+
|Hackney|     Criminal Damage|Criminal Damage T...|    0|2011|    6|
|Hackney|Violence Against ...|          Harassment|    1|2013|    2|
|Hackney|     Criminal Damage|Other Criminal Da...|    0|2011|    7|
|Hackney|Violence Against ...|        Wounding/GBH|    0|2013|   12|
|Hackney|  Theft and Handling|  Other Theft Person|    0|2016|    8|
|Hackney|            Burglary|Burglary in a Dwe...|    2|2008|    5|
|Hackney|             Robbery|   Business Property|    0|2016|    7|
|Hackney|  Theft and Handling|Theft/Taking of P...|    0|2009|   12|
|Hackney|               Drugs|    Drug Trafficking|    0|2014|    4|
|Hackney|  Theft and Handling|Handling Stolen G...|    0|2014|    6|
|Hackney|            Burglary|Burglary in Other...|    0|2008|   12|
|Hackney|Violence Against ...| Ass

In [15]:
### isin method

data_15_16 = data.filter(data["year"].isin(["2015","2016"]))
data_15_16.sample(fraction=0.1).show() 
### sample(fraction) -> samples fraction of the data -> here 10%

+--------------------+--------------------+--------------------+-----+----+-----+
|             borough|      major_category|      minor_category|value|year|month|
+--------------------+--------------------+--------------------+-----+----+-----+
|           Greenwich|Violence Against ...|      Other violence|    0|2016|   11|
|Kingston upon Thames|  Theft and Handling|    Theft From Shops|    0|2016|   11|
|            Hounslow|     Criminal Damage|Criminal Damage T...|    0|2015|    2|
|              Sutton|Violence Against ...|        Wounding/GBH|    0|2016|    7|
|             Lambeth|Violence Against ...|      Other violence|    0|2016|    7|
|              Newham|Violence Against ...|          Harassment|    0|2015|    8|
|           Redbridge|               Drugs| Possession Of Drugs|    2|2016|   11|
|              Bexley|Violence Against ...| Assault with Injury|    0|2015|    6|
|          Wandsworth|     Criminal Damage|Criminal Damage T...|    0|2016|   12|
|           Gree

## Aggregations

In [16]:
borough_crime_count = data.groupBy("borough").count() ## Group by -> aggregation

borough_crime_count.show(5) ## Shows 5

+--------------------+------+
|             borough| count|
+--------------------+------+
|             Croydon|602100|
|          Wandsworth|498636|
|              Bexley|385668|
|             Lambeth|519048|
|Barking and Dagenham|311040|
+--------------------+------+
only showing top 5 rows



In [28]:
### We can do other aggregation on other columns also

borough_conviction_sum = data.groupBy("borough")\
                    .agg({"value":"sum"})\
                    .withColumnRenamed("sum(value)", "conviction")
borough_conviction_sum.show(5)

### Does a sum of the value column Borough wise. -> sum is a built in aggregator

+--------------------+----------+
|             borough|conviction|
+--------------------+----------+
|             Croydon|  260294.0|
|          Wandsworth|  204741.0|
|              Bexley|  114136.0|
|             Lambeth|  292178.0|
|Barking and Dagenham|  149447.0|
+--------------------+----------+
only showing top 5 rows



In [29]:
### If we want acrossn all buroughs: -> without groupby
total_conviction_sum = data\
                    .agg({"value":"sum"})\
                    .withColumnRenamed("sum(value)", "conviction")

In [30]:
total_conviction = total_conviction_sum.collect()[0][0]

## Using functions

In [31]:
import pyspark.sql.functions as func

### functions has some mathematical functions that can be used

borough_percentage_contri = borough_conviction_sum.withColumn(
    "Contribution",
    func.round(borough_conviction_sum.conviction / total_conviction * 100, 2))

borough_percentage_contri.printSchema()

root
 |-- borough: string (nullable = true)
 |-- conviction: double (nullable = true)
 |-- Contribution: double (nullable = true)



In [32]:
borough_percentage_contri.orderBy(borough_percentage_contri[2].desc()).show(10)
### Ordering by the 3rd column descending order

+-------------+----------+------------+
|      borough|conviction|Contribution|
+-------------+----------+------------+
|  Westminster|  455028.0|        7.06|
|      Lambeth|  292178.0|        4.53|
|    Southwark|  278809.0|        4.32|
|       Camden|  275147.0|        4.27|
|       Newham|  262024.0|        4.06|
|      Croydon|  260294.0|        4.04|
|       Ealing|  251562.0|         3.9|
|    Islington|  230286.0|        3.57|
|Tower Hamlets|  228613.0|        3.55|
|        Brent|  227551.0|        3.53|
+-------------+----------+------------+
only showing top 10 rows



In [33]:
crime_category = data.groupBy("major_category").agg({"value":"sum"}).withColumnRenamed("sum(value)", "convictions")

In [34]:
crime_category.orderBy(crime_category.convictions.desc()).show()

+--------------------+-----------+
|      major_category|convictions|
+--------------------+-----------+
|  Theft and Handling|  2661861.0|
|Violence Against ...|  1558081.0|
|            Burglary|   754293.0|
|     Criminal Damage|   630938.0|
|               Drugs|   470765.0|
|             Robbery|   258873.0|
|Other Notifiable ...|   106349.0|
|    Fraud or Forgery|     5325.0|
|     Sexual Offences|     1273.0|
+--------------------+-----------+



In [35]:
year_df = data.select("year")
year_df.agg({"year":"min"}).show()

+---------+
|min(year)|
+---------+
|     2008|
+---------+



In [36]:
year_df.agg({"year":"max"}).show()

+---------+
|max(year)|
+---------+
|     2016|
+---------+



In [37]:
year_df.describe().show()

+-------+------------------+
|summary|              year|
+-------+------------------+
|  count|          13490604|
|   mean|            2012.0|
| stddev|2.5819889931674522|
|    min|              2008|
|    max|              2016|
+-------+------------------+

